# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Windows import
os.chdir('..' + sep_local +'..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..' + sep_local + '..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
/
/home/azeghost/git/azeghost/Generative_Models


In [2]:
print(os.getcwd())

/home/azeghost/git/azeghost/Generative_Models


In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.1


# Dataset loading

In [4]:
dataset_name='atari_pacman'

In [5]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#images_dir = 'C:\\projects\\pokemon\DS06\\'
validation_percentage = 25
valid_format = 'png'

In [6]:
from training.generators.from_images.file_image_generator import create_image_lists, get_generators

In [7]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose=0 
)

In [8]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= image_size
latents_dim = 30
intermediate_dim = 30

In [9]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

  INFO     | Found 1172401 training files

  INFO     | Found 381757 validation files



In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale = 1.0

In [14]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
inputs_shape

(160, 210, 3)

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]


# Model definition

In [17]:
model_name = dataset_name+'GAAE'
experiments_dir='experiments'+sep_local+model_name

In [18]:
from training.adversarial_basic.generative_adversarial.transformative.AAE import AAE as AE

In [19]:
inputs_shape=image_size

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'experiments/atari_pacmanGAAE/var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses
    

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'generative_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size * EPIS_LEN,
    episode_len= 1,
    variables_params=variables_params,
    filepath=None
    )

  WARNING  | None
  WARNING  | None
Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 160, 210, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 160, 210, 15)      60        
_________________________________________________________________
dense_1 (Dense)              (None, 160, 210, 15)      240       
_________________________________________________________________
flatten (Flatten)            (None, 504000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                15120030  
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
_________________________________________________________________
dropout (Dropout)    

In [27]:
#ae.compile(metrics=None)
discr2gen_rate = 0.001
gen2trad_rate = 0.1

ae.compile(
    adversarial_losses=generative_discriminator_losses,
    adversarial_weights={'generator_weight': gen2trad_rate, 'discriminator_weight': discr2gen_rate}
)

Model: "atari_pacmanGAAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 160, 210, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 30)                15120450  
_________________________________________________________________
generative (Functional)      (None, 160, 210, 3)       3126672   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 160, 210, 3)]     0         
Total params: 18,247,122
Trainable params: 18,247,056
Non-trainable params: 66
_________________________________________________________________
None


# Callbacks

In [28]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [30]:
ms = ModelSaver(filepath=_restore)

In [31]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments/atari_pacmanGAAE/csv_dir/atari_pacmanGAAE.csv'

In [32]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [33]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [34]:
import numpy as np

# Model Training

In [36]:
# ae._AA.output_names, ae._AA.input_names

In [37]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=3,
    epochs=2, 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1
)


training <class 'training.autoencoding_basic.transformative.AE.autoencoder'>
Epoch 1/2
3/3 [==============================] - ETA: 0s - loss: 120780.7578 - psnr: 7.9180 - total_variation: 3056132.0000 - ssim_multiscale: 17.0132 - sharpdiff: 5.0987 - mean_absolute_error: 0.3068 - mean_squared_error: 0.1629
Epoch 00001: loss improved from inf to 120780.75781, saving model to experiments/atari_pacmanGAAE/var_save_dir
3/3 [==============================] - 33s 11s/step - loss: 120780.7578 - psnr: 7.9180 - total_variation: 3056132.0000 - ssim_multiscale: 17.0132 - sharpdiff: 5.0987 - mean_absolute_error: 0.3068 - mean_squared_error: 0.1629 - val_loss: 76320.2500 - val_psnr: 10.3238 - val_total_variation: 703788.9375 - val_ssim_multiscale: 33.6460 - val_sharpdiff: 9.4274 - val_mean_absolute_error: 0.2052 - val_mean_squared_error: 0.0952
Epoch 2/2
3/3 [==============================] - ETA: 0s - loss: 55525.1406 - psnr: 11.2020 - total_variation: 3958350.7500 - ssim_multiscale: 52.1829 - sha

ValueError: in user code:

    /home/azeghost/anaconda3/envs/tf-cpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/azeghost/anaconda3/envs/tf-cpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/azeghost/anaconda3/envs/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/azeghost/anaconda3/envs/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/azeghost/anaconda3/envs/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)
